In [1]:
# Imports previos

import matplotlib.pyplot as plt
import nltk
import networkx as nx

from Bio import Medline
from Bio import Entrez

import urllib.request

In [2]:
# Funciones necesarias:

def key_from_value(dict, query):
    for key, values in dict.items():
        for synonym in values:
            if query == synonym:
                return key

def back_to_value(dict, query):
    for keys,values in dict.items():
        for key in keys:
            if query == key:
                return value[0]

# Database para enfermedades

**PLAN RAW PARA ENFERMEDADES**: en Disease Ontology, hay una lista<sup>1</sup> que contiene un ID de la enfermedad, con el id, nombre de la enfermedad y sinónimos. La idea es generar un diccionario  {key: ID y values: nombres de la enfermedad} y si se encuentra en un abstract la enfermedad (es decir, el value), cambiarlo por el ID (la key)
 
 <sup>1</sup>https://github.com/DiseaseOntology/HumanDiseaseOntology/blob/main/src/ontology/HumanDO.obo
 
*Nota*: https://raw.githubusercontent.com/DiseaseOntology/HumanDiseaseOntology/main/src/ontology/HumanDO.obo es el raw

In [3]:
# Importación de lista de enfermedades 

# Se inicializa el diccionario y las variables (inicialmente vacias y false)
disease_dict={}
disease_key=False
disease_name=[]

# Se abre la página del texto raw, se pone todo en minúsculas y se aplica el decode
obo=urllib.request.urlopen('https://raw.githubusercontent.com/DiseaseOntology/HumanDiseaseOntology/main/src/ontology/HumanDO.obo')
obo=[line.lower().decode("utf-8") for line in obo]

# Para cada linea del texto:

# Si empieza con [term] y NO hay definidos una key y value para el diccionario, se inicializan las variables.
# Si empieza con [term] y SÍ hay definidos una key y value para el diccionario, se añaden al diccionario y se resetean

# Si empieza con id, lo convierte en key (disease_key), quitando el "id:", el salto de línea y los espacios en blanco
# Si empieza con name o synonym, lo añade a la lista de values (disease_name) tras eliminar los saltos de línea, el 
#     synonym, el exact [] en el caso de los synonyms, y las comillas.

# Si empieza con [typedef], significa que ya no hay más terms, y que no interesa seguir parseando, con lo que se añade
# el último key-value y se termina

for line in obo:
            
    if line.startswith("[term]"):
        if disease_key and disease_name:
            disease_dict[disease_key]=disease_name
        disease_key=str()
        disease_name=list()
    
    if line.startswith("id"):
        disease_key=line.strip("id:").strip("\n").strip()

    elif line.startswith("name"):
        disease_name.append(line.strip("name:").strip("\n").strip())
    
    elif line.startswith("synonym"):
        disease_name.append(line.strip("\n").strip("synonym: ").strip( "exact []").replace('"',""))

    elif line.startswith("[typedef]"):
        disease_dict[disease_key] = disease_name
        break


In [4]:
print(f'El diccionario de enfermedades contiene {len(disease_dict)} entradas.')
# El resultado es 13046 para la versión del 22 de diciembre de 2020

El diccionario de enfermedades contiene 13046 entradas.


# Database para bacterias

**PLAN RAW PARA BACTERIAS**:  Se obtendrá una lista con el nombre científico de las bacterias, tanto con nombre genérico como epíteto específico (ojo con pasarla a  minúsculas) en un solo string. Una opción es generar un diccionario {key: Nombre científico completo pero todo junto y values: nombre científico separado y nombre abreviado}, y repetir el proceso anterior. Así, todos los nombres científicos (staphylococcus aureus, o s.aureus, hay que recordar que estará todo en minúscula) se cambiarán por el nombre sin espacios 
(staphylococcusaureus o staphylococcus_aureus, es sencillo de lograr de cualquier método) para que al tokenizar sea un solo token. 

He tenido que hacer una cosa un poco fea, poner en el github la lista de los taxids de bacterias (solamente de bacterias, obtenidos a través de un grep del categories.dmp del ftp del NCBI)

Tarda mucho, quizás sería buena idea guardarlo en un archivo JSON?


In [2]:
Entrez.max_tries=5
Entrez.email=""

In [5]:
bact_dict={}

dmp = urllib.request.urlopen("https://raw.githubusercontent.com/GuilleGorines/data/main/b_categories.dmp")
dmp = [line.decode("utf-8").strip("\n").split("\t")[1] for line in dmp]
dmp = set(dmp)

print(f'La cantidad de especies bacterianas encontradas es {len(dmp)}.')

La cantidad de especies bacterianas encontradas es 461159.


In [ ]:
for num,organism in enumerate(dmp):
    search = Entrez.efetch(db='taxonomy',id=organism)
    result = Entrez.read(search)
    result = dict(result[0])
    
    print(result["ScientificName"].lower())
    print(result["LineageEx"][-1])
    print("\n")
    
    


bacteroidales bacterium uba1745
{'TaxId': '185291', 'ScientificName': 'unclassified Bacteroidales', 'Rank': 'no rank'}


lachnospiraceae bacterium uba7481
{'TaxId': '186928', 'ScientificName': 'unclassified Lachnospiraceae', 'Rank': 'no rank'}


pantoea sp. a-su-d-2-7
{'TaxId': '2630326', 'ScientificName': 'unclassified Pantoea', 'Rank': 'no rank'}


microbacterium sp. mdrc-57
{'TaxId': '2609290', 'ScientificName': 'unclassified Microbacterium', 'Rank': 'no rank'}


bacillus sp. hpcaqrksm61
{'TaxId': '185979', 'ScientificName': 'unclassified Bacillus', 'Rank': 'no rank'}


alpha proteobacterium tdb-125
{'TaxId': '33807', 'ScientificName': 'unclassified Alphaproteobacteria', 'Rank': 'no rank'}


flavonifractor sp. an9
{'TaxId': '2629267', 'ScientificName': 'unclassified Flavonifractor', 'Rank': 'no rank'}


rhizobium sp. eecc-221
{'TaxId': '2613769', 'ScientificName': 'unclassified Rhizobium', 'Rank': 'no rank'}


alpha proteobacterium jnvu tp1
{'TaxId': '33807', 'ScientificName': 'uncl

# Generación del corpora con Entrez

In [ ]:
# Se establecen términos de búsqueda

term = "digimon"

Entrez.max_tries=5
Entrez.email=""

In [ ]:
# Búsqueda inicial para comprobar cantidad de coincidencias

scout_search = Entrez.esearch(db="pubmed", rettype="count", term = term)
scout_result = Entrez.read(scout_search)
id_quantity = int(scout_result["Count"])
print(f"{id_quantity} artículo(s) encontrados en PubMed.")

In [ ]:
# Rondas de esearch dividido en rondas de longitud máxima (100k búsquedas)
if id_quantity > 100000:
    rounds = int(id_quantity/100000)+1
    retmax=100*1000
else:
    rounds = 1
    retmax = id_quantity


In [ ]:
# Se inicializa la lista de abstracts 
retstart=0
abstracts=[]

# eFetch y adición de los resultados a la lista de abstracts. 
for round in range(0,rounds):
    
    search = Entrez.esearch(db="pubmed", retmax=retmax, term=term)
    
    id_quantity -= retmax
    retstart += retmax
    if id_quantity < 100000:
        retmax = id_quantity
    result = Entrez.read(search)
    

In [ ]:
for single_id in result['IdList']:
    search = Entrez.efetch(db="pubmed", id=single_id, rettype="medline", retmode="text")
    record = list(Medline.parse(search))
    record = dict(record[0])
    try:
        record = record["AB"]
        abstracts.append(record)
    except KeyError:
        pass


In [ ]:
cantidad_inicial=len(abstracts)

print(f'La cantidad inicial de abstracts es {cantidad_inicial}.')

# Sustitución

In [ ]:
bact_list = [bacteria for sublist in bact_dict.values() for bacteria in sublist]
disease_list = [disease for sublist in disease_dict.values() for disease in sublist]

bacterias_detectadas = []
enfermedades_detectadas = []

coincidencias = []

for num,text in enumerate(abstracts):
    bact_in_text = [num]
    enf_in_text = [num]
    
    for bact in bact_list:
        if bact in text:
            taxid = key_from_value(bact_dict, bact)
            text.replace(bact,taxid)
            bact_in_text.append(taxid)
    
    for disease in disease_list:
        if disease in text:
            doid = key_from_value(disease_dict, disease)
            text.replace(disease, doid)
            enf_in_text.append(doid)
    
    bacterias_detectadas.append(bact_in_text)
    enfermedades_detectadas.append(enf_in_text)